## Lead Scoring Batch Inference Pipeline


In this project, we [UCI Bank Marketing](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing) dataset for lead scoring. Lead classification is used to score leads based on the information we gather. It is different from lead 
qualification which is used to identify ideal customers. Lead scoring helps sales and marketing team focus their efforts on customers who are most likely to buy.

### Configure Workspace
We use mlfow for tracking the experiment and logging metrics and artifacts 

In [1]:
from pyspark.sql import SparkSession
from azureml.core import Workspace, Experiment
import mlflow

# Setup Azure Workspace
ws = Workspace.from_config()
experiment_name = 'leads-pyspark-train'
experiment = Experiment(workspace=ws, name=experiment_name)

# Start MLflow Experiment
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)
run = mlflow.start_run()

# Get Spark session
spark = SparkSession.builder.getOrCreate()

### Generate and Upload Batch Data
We use most of the data for training and testing the model. Rest of the data is used
to create batches by creating csv files with 10 records each. The batches are stored 
in local storage in folder called `batch-data`.

After storing the batches in `batch-data`, the folder is uploaded to blob storage 
using datastore. The dataset is registered as File Dataset. Later on, we can monitor
data drift by creating base and target dataset.

In [2]:
from azureml.core import Dataset
import pandas as pd
import os

df = spark.read.csv(
    path='data/bank-additional-full.csv',
    header="true",
    inferSchema="true",
    sep=";")
trainDF, testDF, batchDF = df.randomSplit([.7, .29, .01], seed=999)
batchData = batchDF.toPandas()

# Create a folder for storing generated batch data
batch_folder = './batch-data'
os.makedirs(batch_folder, exist_ok=True)
print("Folder created!")

# Save each sample as a separate file
print("Saving files...")
x = 0
y = 10
for i in range(int(batchDF.count()/10)):
    filename = str(i+1) + '.csv'
    writeData=batchData[x:y]
    writeData.to_csv(os.path.join(batch_folder, filename), sep=",")
    x+=10
    y+=10

print("files saved!")

# Upload the files to the default datastore
print("Uploading files to datastore...")
default_ds = ws.get_default_datastore()
default_ds.upload(src_dir="batch-data", target_path="batch-data", overwrite=True, show_progress=True)

# Register a dataset for the input data
batch_data_set = Dataset.File.from_files(path=(default_ds, 'batch-data/'), validate=False)
try:
    batch_data_set = batch_data_set.register(workspace=ws, 
                                             name='leads-batch-data',
                                             description='batch data for Marketing Leads UCI',
                                             create_new_version=True)
except Exception as ex:
    print(ex)

print("Done!")

Folder created!
Saving files...
files saved!
Uploading files to datastore...
Uploading an estimated of 40 files
Uploading batch-data/1.csv
Uploaded batch-data/1.csv, 1 files out of an estimated total of 40
Uploading batch-data/10.csv
Uploaded batch-data/10.csv, 2 files out of an estimated total of 40
Uploading batch-data/11.csv
Uploaded batch-data/11.csv, 3 files out of an estimated total of 40
Uploading batch-data/12.csv
Uploaded batch-data/12.csv, 4 files out of an estimated total of 40
Uploading batch-data/13.csv
Uploaded batch-data/13.csv, 5 files out of an estimated total of 40
Uploading batch-data/14.csv
Uploaded batch-data/14.csv, 6 files out of an estimated total of 40
Uploading batch-data/15.csv
Uploaded batch-data/15.csv, 7 files out of an estimated total of 40
Uploading batch-data/16.csv
Uploaded batch-data/16.csv, 8 files out of an estimated total of 40
Uploading batch-data/17.csv
Uploaded batch-data/17.csv, 9 files out of an estimated total of 40
Uploading batch-data/18.cs

### Train Model
There are categorical variables in the dataset which can be either converted into indices or one-hot encoding. Tree based work
better with indicies. So, we have used R formula transformation for SVM and Logistic Regression. For tree based models, 
we have used only numberical columns and indices columns.

A custom Transformer was used to rename columns and to be used along in pipeline. However, there were some problems in integrating
the custom transformer class in scoring script due to which we shifted to normal rename operations with Dataframe.


In [3]:
# A Transformer used in pipelines for renaming columns 
from pyspark.ml import Transformer
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable  


class ColumnRenamer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
    '''
    Renames the following columns in the dataframe: 
    employment variation rate
    consumer price index
    consumer confidence index 
    number of employees 
    '''
    def __init__(self):
        super(ColumnRenamer, self).__init__()
        self.columnsToBeRenamed = {
            'emp.var.rate':'emp_var_rate',
            'cons.price.idx':'cons_price_idx',
            'cons.conf.idx':'cons_conf_idx',
            'nr.employed':'nr_employed'}

    def _transform(self, df):
        for key in self.columnsToBeRenamed.keys():
            df = df.withColumnRenamed(key, self.columnsToBeRenamed[key])
        return df    
rename_columns = ColumnRenamer()


# Uses R Formula for automatic conversion of categorical labels to 1 hot encoding
from pyspark.ml.feature import RFormula
rFormula = RFormula(formula="y ~ .", featuresCol="features", labelCol="label", handleInvalid="skip")

# Uses String Indexer and Numeric Columns only for Tree Based Classifiers
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
example_df = rename_columns.transform(trainDF)
categorialColumns = [colname for (colname, dataType) in example_df.dtypes if ((dataType=="string") and (colname!="y"))]
stringIndexer = StringIndexer(inputCols=categorialColumns, outputCols=[c + "Index" for c in categorialColumns])
oheEncoder = OneHotEncoder(inputCols=stringIndexer.getOutputCols(), outputCols=[c + "ohe" for c in categorialColumns])
label_stringIdx = StringIndexer(inputCol="y", outputCol="label")
numericColumns = [colname for (colname, dataType) in example_df.dtypes if (dataType=="int" or dataType=="float" or dataType=="double")]
assembledInputs = numericColumns + [c + "Index" for c in categorialColumns]
vecAssembler = VectorAssembler(inputCols=assembledInputs, outputCol="features")

Models are trained and stored in local path in folder called `model`. Then they are registered in AML workspace. We use Reciever Operating Characteristics
(ROC) curve, Precision Recall curve and accuracy for evaluating the models.

We log the models and metrics using MLflow.

In [4]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
import mlflow.spark
import pandas as pd

# For Tracking Models
model_num=1
pipelineModel = None

# Evaluators for performance metrics
bevaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label")
mevaluator = MulticlassClassificationEvaluator()

# Rename Columns
columnsToBeRenamed = {'emp.var.rate':'emp_var_rate','cons.price.idx':'cons_price_idx','cons.conf.idx':'cons_conf_idx','nr.employed':'nr_employed'}
for key in columnsToBeRenamed.keys():
    trainDF = trainDF.withColumnRenamed(key, columnsToBeRenamed[key])
for key in columnsToBeRenamed.keys():
    testDF = testDF.withColumnRenamed(key, columnsToBeRenamed[key])

# Non Tree Based Models
non_tree_models = [LogisticRegression(), LinearSVC()]
for model in non_tree_models:
    non_tree_pipeline = Pipeline(stages=[rFormula, model])
    pipelineModel = non_tree_pipeline.fit(trainDF)
    predDF = pipelineModel.transform(testDF)

    modelName =str(model_num)+'-'+model.__class__.__name__
    accuracy = mevaluator.setMetricName("accuracy").evaluate(predDF)
    roc = bevaluator.setMetricName("areaUnderROC").evaluate(predDF)
    pr = bevaluator.setMetricName("areaUnderPR").evaluate(predDF)
    model_num += 1


    # Log metrics and model
    mlflow.spark.log_model(pipelineModel, modelName)
    mlflow.log_metrics({"modelNum":model_num, "accuracy":accuracy, "areaUnderROC":roc, "areaUnderPR":pr})
    print("Training complete:",modelName)

# Tree Based Models
tree_models = [DecisionTreeClassifier(), RandomForestClassifier(), GBTClassifier()]
for model in tree_models:
    tree_pipeline = Pipeline(stages=[stringIndexer, oheEncoder, label_stringIdx, vecAssembler,model])
    pipelineModel = tree_pipeline.fit(trainDF)
    predDF = pipelineModel.transform(testDF)

    modelName = str(model_num)+'-'+model.__class__.__name__
    accuracy = mevaluator.setMetricName("accuracy").evaluate(predDF)
    roc = bevaluator.setMetricName("areaUnderROC").evaluate(predDF)
    pr = bevaluator.setMetricName("areaUnderPR").evaluate(predDF)
    model_num += 1

    # Log metrics and model
    mlflow.spark.log_model(pipelineModel, modelName)
    mlflow.log_metrics({"modelNum":model_num, "accuracy":accuracy, "areaUnderROC":roc, "areaUnderPR":pr})
    print("Training complete:",modelName)


pipelineModel.save('model')

from azureml.core import Model
Model.register(
    workspace=ws,
    model_path='model/',
    model_name='pyspark-batch-leads-model',
)

mlflow.end_run()

Training complete: 1-LogisticRegression
Training complete: 2-LinearSVC
Training complete: 3-DecisionTreeClassifier
Training complete: 4-RandomForestClassifier
Training complete: 5-GBTClassifier
Registering model pyspark-batch-leads-model


### Create Compute
We create compute which is to be used for running batch inference jobs.

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Compute params
compute_name = 'rohan-vm-cluster'
inference_cluster = None

if compute_name in ws.compute_targets:
    inference_cluster = ComputeTarget(ws, compute_name)
    print("Using existing cluster.")
else:
    try:
        compute_config = AmlCompute.provisioning_configuration(
            vm_size ='STANDARD_DS11_V2', 
            max_nodes=2 )
        inference_cluster = ComputeTarget.create(ws, compute_name, compute_config)
        inference_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    print("Cluster created.")

Using existing cluster.


### Scoring Script
We create a folder called `batch-pipeline` to save `environment.yml` file and `scoring script` for the pipeline.

In [6]:
# Create a folder for storing batch-pipeline files
batch_folder = './batch-pipeline'
os.makedirs(batch_folder, exist_ok=True)
print("Folder created!")

Folder created!


In [7]:
%%writefile 'batch-pipeline/batch_environment.yml'
name: mlflow-environment
channels:
  - defaults
  - anaconda
  - conda-forge
dependencies:
  - python=3.6
  - scikit-learn
  - pip
  - pandas
  - numpy
  - openjdk
  - pip:
    - pyspark
    - mlflow
    - azureml-mlflow
    - azureml-defaults

Writing batch-pipeline/batch_environment.yml


In [8]:
%%writefile 'batch-pipeline/score.py'
import os
import numpy as np
from azureml.core import Model
from pyspark.ml import PipelineModel
from pyspark.sql import SparkSession

def init():
    global model, columnsToBeRenamed, spark
    spark = SparkSession.builder.getOrCreate()
    model_path = Model.get_model_path('pyspark-batch-leads-model')
    model= PipelineModel.load(model_path)
    columnsToBeRenamed = {'emp.var.rate':'emp_var_rate','cons.price.idx':'cons_price_idx','cons.conf.idx':'cons_conf_idx','nr.employed':'nr_employed'}

def run(mini_batch):
    # This runs for each batch
    resultList = []
    # process each file in the batch
    for f in mini_batch:
        df = spark.read.csv(path=f,header="true",inferSchema="true",sep=",").drop('_c0')
        for key in columnsToBeRenamed.keys():
            df = df.withColumnRenamed(key, columnsToBeRenamed[key]) 
        prediction = model.transform(df).select('prediction').toPandas().prediction.map({0.0:"no",1.0:"yes"}).to_numpy()
        resultList.append("{}: {}".format(os.path.basename(f), prediction))
    return resultList


Writing batch-pipeline/score.py


### Create Pipeline
We create a batch inference pipeline with ParallelRunStep. ParallelRunStep parallelizes batch operations.

In [9]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# Create an Environment for the experiment
batch_env = Environment.from_conda_specification(name="experiment_env", file_path="batch-pipeline/batch_environment.yml")
batch_env.docker.base_image = DEFAULT_CPU_IMAGE
print('Configuration ready.')

Configuration ready.


In [10]:
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.data import OutputFileDatasetConfig
from azureml.core.runconfig import DockerConfiguration

output_dir = OutputFileDatasetConfig(name='inferences')

parallel_run_config = ParallelRunConfig(
    source_directory='batch-pipeline/',
    entry_script="score.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=inference_cluster,
    node_count=2)

parallelrun_step = ParallelRunStep(
    name='batch-score-leads',
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('leads_batch')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

print('Steps defined')

Steps defined


In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
pipeline_run = Experiment(workspace=ws, name='leads-batch-pipeline').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)



### Fetch predictions from run
A batch inference pipeline coelsce data from multiple nodes in cluster to a single file called `inference` in the pipeline's output.

The result of batch inference pipeline is downloaded from the blob storage to local storage
in `leads-results` folder. We look for the file `parallel_run_step.txt` in `leads-results` and load it as pandas dataframe.
Later on, we write the final leads as csv in `leads-results` folder.

In [12]:
import pandas as pd
import shutil

# Remove the local results folder if left over from a previous run
result_file = None
shutil.rmtree('leads-results', ignore_errors=True)

# Get the run for the first step and download its output
prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='leads-results')

# Traverse the folder hierarchy and find the results file
for root, dirs, files in os.walk('leads-results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]
df.to_csv("leads-results/leads.csv", header=True,index=False)
# Display the first 20 results
df.head(20)

,File,Prediction
0,1.csv,['no' 'no' 'yes' 'no' 'no' 'yes' 'no' 'no' 'n...
1,10.csv,['no' 'yes' 'no' 'no' 'no' 'no' 'no' 'no' 'no...
2,11.csv,['no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no'...
3,12.csv,['no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no'...
4,13.csv,['no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no'...
5,14.csv,['no' 'yes' 'no' 'no' 'no' 'no' 'no' 'no' 'no...
6,15.csv,['no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no'...
7,16.csv,['no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no'...
8,17.csv,['yes' 'no' 'no' 'yes' 'no' 'no' 'no' 'no' 'n...
9,18.csv,['no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no'...


### Publish Pipeline
The pipeline is published online so that it can be consumed by user with the help of authentication toke.

In [13]:
published_pipeline = pipeline_run.publish_pipeline(name='leads-batch-pipeline', description='Batch scoring of leads data from UCI', version='1.0')
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://centralindia.api.azureml.ms/pipelines/v1.0/subscriptions/23416925-66df-470c-b651-f378856d8ad7/resourceGroups/rohan-rg/providers/Microsoft.MachineLearningServices/workspaces/rohan-ws/PipelineRuns/PipelineSubmit/eedecb53-a277-43d7-a5ef-f31e71dba0b0


### Test the Pipeline
We use HTTP request to run the deployed pipeline.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails
import requests

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, headers=auth_header, json={"ExperimentName": "leads-batch-pipeline"})
run_id = response.json()["Id"]

published_pipeline_run = PipelineRun(ws.experiments['leads-batch-pipeline'], run_id)
published_pipeline_run.wait_for_completion(show_output=False)

print("OK")